In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

ModuleNotFoundError: No module named 'torch'

In [ ]:
# 設定 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#### TODO
data = pd.read_csv('/data/winequality-red_train.csv', delimiter=',')
#### TODO

selected_features = True
if selected_features:
  #### TODO

  #### TODO
if selected_features:
  data = data[selected_features + ['quality']]

# 分割特徵與標籤
X = data.iloc[:, :-1].values  # 特徵
y = data.iloc[:, -1].values   # 標籤 (Wine Quality)

# 標準化數據
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
# 切分訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 轉換為 PyTorch Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# 創建 DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# 定義神經網絡模型
class WineQualityNN(nn.Module):
    def __init__(self, input_dim):
        super(WineQualityNN, self).__init__()
        #### TODO
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(32, )  # Wine quality 範圍為 0-10
        #### TODO

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# 初始化模型
input_dim = X.shape[1]
model = WineQualityNN(input_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 訓練模型
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
# 測試模型
def test_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# 執行訓練與測試
train_model(model, train_loader, criterion, optimizer, epochs=100)
test_model(model, test_loader)

In [ ]:
# 預測新數據並保存到同一個 CSV
def predict_and_save_combined(model, selected_features, files, output_csv):
    results = []
    for file_path, wine_type in files:
        data = pd.read_csv(file_path, delimiter=',')
        if selected_features:
            data = data[selected_features]
        X_new = scaler.transform(data.values)
        X_new_tensor = torch.tensor(X_new, dtype=torch.float32).to(device)

        with torch.no_grad():
            outputs = model(X_new_tensor)
            _, predicted = torch.max(outputs, 1)

        results.extend([
            {'ID': f"{wine_type}_{i+1}", 'quality': int(pred.cpu().numpy())}
            for i, pred in enumerate(predicted)
        ])

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv, index=False)
    print(f"Predictions saved to {output_csv}")

In [ ]:
# 預測紅酒與白酒品質，合併輸出至單一 CSV
predict_and_save_combined(model,
  selected_features,
 [("winequality-red_goal.csv", "red"), ("winequality-white_goal.csv", "white")],
                          "/content/baseline_winequality_predictions.csv")